In [2]:
# 케라스 튜너 사용하기
!pip install keras-tuner -q

In [3]:
# 모델 구축 함수 정의
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
  units = hp.Int(name = 'units', min_value = 16, max_value = 64, step = 16)
  model = keras.Sequential([
      layers.Dense(units, activation = 'relu'),
      layers.Dense(10, activation = 'softmax')
  ])

  optimizer = hp.Choice(name = 'optimizer', values = ['rmsprop', 'adam'])
  model.compile(optimizer = optimizer,
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model


In [4]:
# 더 쉽게 만들기  : HyperModel 클래스 상속

import keras_tuner as kt

class SimpleMLP(kt.HyperModel):
  def __init__(self, num_classes):
    self.num_classes = num_classes

  def build(self, hp):
    units = hp.Int(name = 'units', min_value = 16, max_value = 64, step = 16)
    model = keras.Sequential([
        layers.Dense(units, activation = 'relu'),
        layers.Dense(self.num_classes, activation = 'softmax')
    ])
    optimizer = hp.Choice(name = 'optimizer', values = ['rmsprop', 'adam'])

    model.compile(optimizer = optimizer,
                  loss = 'sparse_categorical_crossentropy',
                  metrics = ['accuracy'])

    return model

hypermodel = SimpleMLP(num_classes = 10)

In [5]:
# 튜너 정의하기
tuner = kt.BayesianOptimization(
    hypermodel, # 모델 구축 함수 or HyperModel 클래스 객체
    objective = 'val_accuracy', # 튜너가 최적화할 지표 : 항상 검증 지표를 지정하자!
    max_trials = 25, # 하이퍼파라미터 탐색 공간에서 선택하는 횟수
    executions_per_trial = 2, # 같은 하이퍼파라미터로 훈련하는 횟수. 검증은 평균값을 이용함.
    directory = 'mnist_kt_test', # 탐색 로그 저장 위치

    overwrite = True  # 디렉토리 데이터를 덮어쓸 것인가
    # 모델 구축 함수를 수정했다면 True, 같은 모델 구축 함수로 탐색을 잇는다면 False
)

In [7]:
# 탐색 시작
# 검증 데이터를 전달한다. 테스트 데이터는 하이퍼파라미터 튜닝 후에 적용한다.

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype('float32') / 255
x_test = x_test.reshape((-1, 28 * 28)).astype('float32') / 255

# 나중에 쓰기 위해 따로 보관함
x_train_full = x_train[:]
y_train_full = y_train[:]

num_val_samples = 10000
x_train, x_val= x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val= y_train[:-num_val_samples], y_train[-num_val_samples:]

# 모델마다 얼마나 많은 에포크가 필요한지 모르기 때문에
# 에포크는 크게 지정하고, 과대적합 시작 전에 훈련을 멈춘다.
callbacks = [
    keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)
]

# fit과 동일한 매개변수를 사용한다.
# 모델의 fit() 메서드마다 이 매개변수들이 전달된다.
tuner.search(x_train,
             y_train,
             batch_size = 128,
             epochs = 100,
             validation_data = (x_val, y_val),
             callbacks = callbacks,
             verbose = 2)

In [8]:
# 최상의 하이퍼파라미터 설정 확인하기
top_n = 4

# 모델 구축 함수에 전달할 수 있는 HyperParameters 객체 리스트 전달
best_hps = tuner.get_best_hyperparameters(top_n)

In [9]:
# 최상의 에포크 찾기
def get_best_epoch(hp):
  model = build_model(hp)
  callbacks = [
      keras.callbacks.EarlyStopping(
          monitor = 'val_loss',
          mode = 'min',
          patience = 10
      )
  ]
  history = model.fit(x_train, y_train,
                      validation_data = (x_val, y_val),
                      epochs = 100,
                      batch_size = 128,
                      callbacks = callbacks)
  val_loss_per_epoch = history.history['val_loss']
  best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
  print(f"최상의 에포크 : {best_epoch}")
  return best_epoch


In [10]:
# 최적 모델과 에포크를 얻고, 더 많은 데이터에서 조금 더 오래 훈련한다.
def get_best_trained_model(hp):

  best_epoch = get_best_epoch(hp)
  model = build_model(hp) # 책에는 이게 빠진 듯?

  model.fit(x_train_full, y_train_full,
            batch_size = 128, epochs = int(best_epoch * 1.2))
  return model

In [11]:
# 실행 부분
best_models = []
for hp in best_hps:
  model = get_best_trained_model(hp)
  model.evaluate(x_test, y_test)
  best_models.append(model)

Epoch 1/100
391/391 [==============================] - 2s 4ms/step - loss: 0.4270 - accuracy: 0.8861 - val_loss: 0.2404 - val_accuracy: 0.9344
Epoch 2/100
391/391 [==============================] - 1s 4ms/step - loss: 0.2176 - accuracy: 0.9382 - val_loss: 0.1813 - val_accuracy: 0.9495
Epoch 3/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1645 - accuracy: 0.9529 - val_loss: 0.1572 - val_accuracy: 0.9554
Epoch 4/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1347 - accuracy: 0.9616 - val_loss: 0.1300 - val_accuracy: 0.9640
Epoch 5/100
391/391 [==============================] - 1s 4ms/step - loss: 0.1140 - accuracy: 0.9676 - val_loss: 0.1161 - val_accuracy: 0.9668
Epoch 6/100
391/391 [==============================] - 1s 4ms/step - loss: 0.0989 - accuracy: 0.9719 - val_loss: 0.1121 - val_accuracy: 0.9671
Epoch 7/100
391/391 [==============================] - 2s 6ms/step - loss: 0.0874 - accuracy: 0.9752 - val_loss: 0.1032 - val_accuracy: 0.9707

## 단일 호스트, 다중 동기 훈련

In [12]:
import tensorflow as tf
 # 분산 전략 객체 : 디폴트값임
strategy = tf.distribute.MirroredStrategy()
print(f"장치 개수 : {strategy.num_replicas_in_sync}")

 # 모든 변수는 이 코드블록 안에서 만들어진다. 모델 구축과 compile까지.
# with strategy.scope():
# 	model = get_compiled_model()
# model.fit(train_dataset,
# 		 epochs=100,
# 		 validation_data = val_dataset,
# 		 callbacks = callbacks)

장치 개수 : 1


## TPU

In [1]:
# 코랩에서 TPU 사용하기 - 런타임에서 tpu 켜자

# 이건 그냥 무지성으로 켜면 된다.
# import tensorflow as tf

# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
# print("장치 : ", tpu.master())

장치 :  grpc://10.73.203.106:8470


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
print("장치 : ", tpu.master())

strategy = tf.distribute.TPUStrategy(tpu)
print("복제 모델 개수 : ", strategy.num_replicas_in_sync)

def build_model(input_size):
  inputs = keras.Input((input_size, input_size, 3))
  x = keras.applications.resnet.preprocess_input(inputs)
  x = keras.applications.resnet.ResNet50(weights = None, include_top = False,
                                         pooling = 'max')(x)
  outputs = layers.Dense(10, activation = 'softmax')(x)
  model = keras.Model(inputs, outputs)
  model.compile(optimizer = 'rmsprop',
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy']
                )
  return model

with strategy.scope():
  model = build_model(input_size = 32)

장치 :  grpc://10.73.203.106:8470
복제 모델 개수 :  8


In [6]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
model.fit(x_train, y_train, batch_size = 1024) # TPU 훈련 시 다중 GPU 훈련처럼 배치 크기가 충분히 커야 한다.

49/49 [==============================] - 69s 488ms/step - loss: 2.7354 - accuracy: 0.2619
